In [41]:
import pandas as pd
import numpy as np

df = pd.read_csv('/home/rmartinez/Documents/github_projects/mexico_earthquakes/data/sismos.csv', encoding='latin1')


df.replace('en revision', np.nan, inplace=True)
df.replace('no calculable', np.nan, inplace=True)
df['Fecha_Hora']=pd.to_datetime(df['Fecha'] + ' '+ df['Hora'])
df_labels = pd.DataFrame(df['Profundidad'],columns=['Profundidad'])
df.drop(columns=['Fecha','Hora','Fecha_UTC','Hora_UTC','Estatus','Profundidad'], inplace=True, axis=1)
df_num = df.drop(['Localizacion','Fecha_Hora'], axis=1)


/home/rmartinez/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
import re
regex = '^.*km al .*de .*,'

local_sub = lambda x: re.sub(regex,'',x)

df['Localizacion']=df['Localizacion'].apply(local_sub)
df['Localizacion'].value_counts()

median = df['Magnitud'].median()
df['Magnitud'].fillna(median,inplace=True)
df_num_attr =list(df_num)
df_cat_attr = ['Localizacion']

median = df_labels['Profundidad'].median()
df_labels['Profundidad'].fillna(median, inplace=True)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import CategoricalEncoder

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

num_pipeline = Pipeline([('selector',DataFrameSelector(df_num_attr)),('imputer',Imputer(strategy='median')),])
cat_pipeline = Pipeline([('selector',DataFrameSelector(df_cat_attr)),('cat_encoder',CategoricalEncoder(encoding='onehot-dense')),])
full_pipeline = FeatureUnion(transformer_list=[('num_pipeline',num_pipeline),('cat_pipeline',cat_pipeline),])

earthquakes_prepared = full_pipeline.fit_transform(df)
earthquakes_prepared

#imputer = Imputer(strategy='median')
##imputer.fit(df_num)
#imputer.statistics_
##df_num = pd.DataFrame(imputer.transform(df_num), columns=df_num.columns)

/home/rmartinez/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[   7.4 ,   20.  , -105.  , ...,    0.  ,    0.  ,    0.  ],
       [   6.9 ,   20.  , -105.  , ...,    0.  ,    0.  ,    0.  ],
       [   6.9 ,   25.  , -110.  , ...,    0.  ,    0.  ,    0.  ],
       ...,
       [   2.4 ,   19.19, -104.24, ...,    0.  ,    0.  ,    0.  ],
       [   3.3 ,   19.34, -104.49, ...,    0.  ,    0.  ,    0.  ],
       [   3.5 ,   16.39,  -94.91, ...,    0.  ,    0.  ,    0.  ]])

In [43]:
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()
linear_reg.fit(earthquakes_prepared,df_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [44]:
some_data = df.iloc[:5]
some_labels = df_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

linear_reg.predict(some_data_prepared)

array([[69.19049072],
       [64.75317383],
       [34.12261963],
       [50.04803467],
       [67.28930664]])

In [46]:
from sklearn.metrics import mean_squared_error

earthquake_predictions = linear_reg.predict(earthquakes_prepared)
lin_mse = mean_squared_error(earthquake_predictions, df_labels)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

29.047941457955023

In [53]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(earthquakes_prepared,df_labels)
tree_predictions = tree_reg.predict(earthquakes_prepared)
lin_mse = mean_squared_error(tree_predictions, df_labels)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

3.564526993046849

In [99]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, earthquakes_prepared, df_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print('Scores:',scores)
    print('Mean:',scores.mean())
    print('Standard Deviation:',scores.std())

display_scores(tree_rmse_scores)

Scores: [35.83259045 37.62559777 32.36669069 31.00024146 28.80691485 27.46751689
 26.98123593 27.13395414 30.42034906 25.1402202 ]
Mean: 30.277531144457235
Standard Deviation: 3.8335173993056


In [100]:
scores_lin = cross_val_score(linear_reg, earthquakes_prepared, df_labels, scoring='neg_mean_squared_error', cv=10)
linear_reg_scores = np.sqrt(-scores_lin)
display_scores(linear_reg_scores)

Scores: [3.32294940e+01 3.24719635e+01 3.34129774e+01 3.17222660e+01
 2.91507788e+01 2.92380617e+01 2.81417752e+01 1.92700292e+10
 2.75527239e+01 3.64726376e+09]
Mean: 2291729315.454199
Standard Deviation: 5762953442.937418


In [102]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(earthquakes_prepared, df_labels)

scores_forest = cross_val_score(forest_reg, earthquakes_prepared, df_labels, scoring="neg_mean_squared_error", cv=10)
forest_reg_scores = np.sqrt(-scores_forest)
display_scores(forest_reg_scores)

/home/rmartinez/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
/home/rmartinez/.local/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:487: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/rmartinez/.local/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:487: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/rmartinez/.local/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:487: DataConversionWarning: A column-vector

Scores: [28.45932675 31.39535657 26.17381596 24.03591164 21.78356821 21.34947329
 20.44702972 20.31441021 24.13224537 19.57545048]
Mean: 23.766658817659113
Standard Deviation: 3.6945681380095414
